In [2]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset preprocessing

In [4]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb_paired.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb.csv')
mcpas_raw = pd.read_csv('data/raw/mcpas-tcr.csv', encoding='latin1')

/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_13121/3174298451.py:2: DtypeWarning: Columns (8,10,11,15,16,17,18,19,20,21,22,23,25,30,31,36,37,44,45,46,47,48,49,50,51,52,54,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb_raw = pd.read_csv('data/raw/iedb.csv')
/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_13121/3174298451.py:3: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mcpas_raw = pd.read_csv('data/raw/mcpas-tcr.csv', encoding='latin1')


## VDJdb

In [44]:
vdj_cols = ['complex.id', 'CDR3', 'V', 'J', 'Species', 'MHC A','MHC class', 'Epitope']
print('VDJdb raw:', vdjdb_raw.shape)
vdjdb_a = vdjdb_raw[vdjdb_raw['Gene']=='TRA'][vdj_cols].copy()
vdjdb_b = vdjdb_raw[vdjdb_raw['Gene']=='TRB'][vdj_cols].copy()

vdjdb = pd.merge(vdjdb_a, vdjdb_b, on='complex.id')
print('VDJdb paired:', vdjdb.shape)

assert (vdjdb['Species_x'] == vdjdb['Species_y']).all()
assert (vdjdb['Epitope_x'] == vdjdb['Epitope_y']).all()
assert (vdjdb['MHC A_x'] == vdjdb['MHC A_y']).all()

vdj_cols = ['complex.id', 'CDR3_x', 'V_x', 'J_x', 'CDR3_y', 'V_y', 'J_y', 'Species_y', 'MHC A_y', 'MHC class_y', 'Epitope_y']
vdjdb = vdjdb[vdj_cols].copy()
vdj_cols = ['complex.id', 'cdr3a', 'va', 'ja', 'cdr3b', 'vb', 'jb', 'species', 'mhc', 'mhc_class', 'epitope']
vdjdb.columns = vdj_cols
vdjdb.head()


VDJdb raw: (2502, 17)
VDJdb paired: (1251, 15)


,complex.id,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,15,CAVAGYGGSQGNLIF,TRAV12-2*01,TRAJ42*01,CASSPQGLGTEAFF,TRBV28*01,TRBJ1-1*01,HomoSapiens,HLA-A*02,MHCI,ELAGIGILTV
1,103,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:59,MHCI,LLFGYPVYV
2,104,CAAMEGAQKLVF,TRAV13-1*01,TRAJ54*01,CASSYPGGGFYEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:48,MHCI,LLFGYPVYV
3,114,CAGAGSQGNLIF,TRAV27*01,TRAJ42*01,CASSSRSSYEQYF,TRBV19*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:48,MHCI,GILGFVFTL
4,115,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:59,MHCI,LLFGYAVYV


In [43]:
# vdjdb = vdjdb.drop('complex.id', axis=1)
# print('unique TCR-pMHC complex:', vdjdb.shape)
# complex = []
# for i in range(len(vdjdb)):
#     complex.append((vdjdb['cdr3a'].iloc[i], vdjdb['cdr3b'].iloc[i], vdjdb['epitope'].iloc[i]))
# vdjdb['complex'] = complex
# vdjdb_dict = vdjdb.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
# print('unique tcr-peptide pairs:', len(vdjdb_dict.keys()))

In [51]:
# final data formating
try:
    vdjdb = vdjdb.drop('complex.id', axis=1)
except:
    pass
vdjdb = vdjdb.replace('MHCI', 1) # name formatting
vdjdb = vdjdb.replace('HomoSapiens', 'human') # name formatting
vdjdb = vdjdb[vdjdb['species']=='human'].copy() # keep only humans
vdjdb = vdjdb.drop_duplicates()
print("preprocessed vdjdb:", vdjdb.shape)
vdjdb.head()

preprocessed vdjdb: (967, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,CAVAGYGGSQGNLIF,TRAV12-2*01,TRAJ42*01,CASSPQGLGTEAFF,TRBV28*01,TRBJ1-1*01,human,HLA-A*02,1,ELAGIGILTV
1,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:59,1,LLFGYPVYV
2,CAAMEGAQKLVF,TRAV13-1*01,TRAJ54*01,CASSYPGGGFYEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:48,1,LLFGYPVYV
3,CAGAGSQGNLIF,TRAV27*01,TRAJ42*01,CASSSRSSYEQYF,TRBV19*01,TRBJ2-7*01,human,HLA-A*02:01:48,1,GILGFVFTL
4,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:59,1,LLFGYAVYV


## IEDB

IEDB queried for MHC 1 and human only

In [39]:
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb = iedb_raw.dropna(subset=subset)
iedb = iedb.astype('str')
print("Full IEDB:", iedb.shape)

iedb_base_cols = ['Group Receptor ID', 'Receptor ID', 'Description', 'Organism', 'MHC Allele Names']

iedb_cols = [col for col in iedb.columns if "Curated" in col and ("CDR3" in col or "V" in col or "J" in col)]
iedb_cols = ['Description', 'Organism', 'MHC Allele Names', 'Curated Chain 1 V Gene', 'Curated Chain 1 J Gene', 'Chain 1 CDR3 Curated',  
            'Curated Chain 2 V Gene','Curated Chain 2 J Gene', 'Chain 2 CDR3 Curated']
iedb = iedb[iedb_cols]
print("IEDB Selected Columns:", iedb.shape)

iedb = iedb.replace('nan', np.nan)
iedb = iedb.dropna()
iedb

Full IEDB: (26710, 71)
IEDB Selected Columns: (26710, 9)


,Description,Organism,MHC Allele Names,Curated Chain 1 V Gene,Curated Chain 1 J Gene,Chain 1 CDR3 Curated,Curated Chain 2 V Gene,Curated Chain 2 J Gene,Chain 2 CDR3 Curated
93,SLLMWITQC,Homo sapiens (human),HLA-A*02:01,TRAV21*01,TRAJ6*01,CAVRPTSGGSYIPTF,TRBV6-5*01,TRBJ2-2*01,CASSYVGNTGELFF
187,AAGIGILTV,Homo sapiens (human),HLA-A*02:01,TRAV35*02,TRAJ49*01,CAGGTGNQFYF,TRBV10-3,TRBJ1-5*01,CAISEVGVGQPQHF
189,AAGIGILTV,Homo sapiens (human),HLA-A*02:01,TRAV35*02,TRAJ49*01,CAGGTGNQFYF,TRBV10-3,TRBJ1-5*01,CAISEVGVGQPQHF
216,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),HLA-A*02:01,TCRAV3*01,TCRAJ39*01,AVNYGNMLT,TCRBV27*01,TCRBJ2-1*01,ASSKTSGSPYNEQF
217,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),HLA-A*02:01,TCRAV26-2,TCRAJ43,ILDNNNDMR,TCRBV7-6,TCRBJ1-4,ASSLAPGTTNEKLF
...,...,...,...,...,...,...,...,...,...
27059,KPVETSNSF,SARS-CoV2,HLA-B*07:02,TRAV21,TRAJ58,CGAGETSGSRLTF,TRBV29-1,TRBJ2-3,CSVNLGGPTDTQYF
27060,TTDPSFLGRY,SARS-CoV2,HLA-A*01:01,TRAV9-2,TRAJ42,CALEGSQGNLIF,TRBV20-1,TRBJ1-2,CSVPDGAEPYGYTF
27061,SPRWYFYYL,SARS-CoV2,HLA-B*07:02,TRAV4,TRAJ9,CLVGNTGGFKTIF,TRBV29-1,TRBJ1-1,CSVPDRGNTEAFF
27063,VSDGGPNLY,Influenza A virus,HLA-A*01:01,TRAV9-2,TRAJ38,CAPSRHAGNNRKLIW,TRBV29-1,TRBJ1-4,CSVQGGTNEKLFF


In [52]:
iedb.columns

Index(['Description', 'Organism', 'MHC Allele Names', 'Curated Chain 1 V Gene',
       'Curated Chain 1 J Gene', 'Chain 1 CDR3 Curated',
       'Curated Chain 2 V Gene', 'Curated Chain 2 J Gene',
       'Chain 2 CDR3 Curated'],
      dtype='object')